# JSTOR Data Processing and Cleaning

This notebook processes and cleans the full JSTOR data title, primarily focused on processing dates, establishing publication lineage, binning data, adding in wiki data, and creating a final dataframe for analysis.

### Load Libraries

In [2]:
import pandas as pd
import numpy as np
import altair as alt
alt.renderers.enable("mimetype")
alt.data_transformers.enable('default', max_rows=None)
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

## JSTOR All Journals

### Load and Clean Data

In [3]:
global_txt_file = pd.read_csv("../data/original_journal_datasets/jstor/jstor_titles/JSTOR_Global_AllArchiveTitles_2024-02-21.txt", sep="\t")

In [138]:
len(global_txt_file)

4497

#### Trace Lineage and Parent Publications

In [4]:
global_txt_file['grouped_id'] = global_txt_file.preceding_publication_title_id.fillna(global_txt_file.title_id)
# Function to trace the lineage of a publication
def trace_lineage(title_id, df):
    preceding_id = df[df['title_id'] == title_id]['preceding_publication_title_id'].iloc[0]
    if pd.isna(preceding_id):
        return title_id
    else:
        return trace_lineage(preceding_id, df)

tqdm.pandas(desc="Tracing Lineage")
# Apply the function to create a new column for lineage
global_txt_file['lineage'] = global_txt_file['title_id'].progress_apply(lambda x: trace_lineage(x, global_txt_file))

# Function to map each publication to its parent
def map_parent(title_id, df):
    parent_id = df[df['title_id'] == title_id]['parent_publication_title_id'].iloc[0]
    return parent_id if not pd.isna(parent_id) else title_id

tqdm.pandas(desc="Mapping Parent")
# Apply the function to create a new column for parent
global_txt_file['parent'] = global_txt_file['title_id'].progress_apply(lambda x: map_parent(x, global_txt_file))

Tracing Lineage:   0%|          | 0/4497 [00:00<?, ?it/s]

Mapping Parent:   0%|          | 0/4497 [00:00<?, ?it/s]

#### Process Dates 

Need to deal with earlier than 1677 dates

In [5]:
def conv(date):
    year, month, day = map(int, date.split("-"))
    return pd.Period(year=year, month=month, day=day, freq="D")

tqdm.pandas(desc="Processing date_first_issue_online")
global_txt_file["processed_date_first_issue_online"] = global_txt_file.date_first_issue_online.progress_apply(conv)

tqdm.pandas(desc="Processing date_last_issue_online")
global_txt_file["processed_date_last_issue_online"] = global_txt_file.date_last_issue_online.progress_apply(conv)

Processing date_first_issue_online:   0%|          | 0/4497 [00:00<?, ?it/s]

Processing date_last_issue_online:   0%|          | 0/4497 [00:00<?, ?it/s]

#### Infer length of publications

In [6]:
def infer_lineage_length_publication(rows):
    total_active_years = 0

    for _, row in rows.iterrows():
        # Check if both dates are not null
        if pd.notnull(row['processed_date_first_issue_online']) and pd.notnull(row['processed_date_last_issue_online']):
            start_year = row['processed_date_first_issue_online'].year
            end_year = row['processed_date_last_issue_online'].year
            active_years = end_year - start_year + 1  # +1 to include both start and end year
            total_active_years += active_years

    # Apply the total active years to each row in the group
    rows['lineage_active_years'] = total_active_years

    return rows

tqdm.pandas(desc="Infer length of publication by lineage")
global_txt_file = global_txt_file.groupby('lineage').progress_apply(infer_lineage_length_publication)

Infer length of publication by lineage:   0%|          | 0/2851 [00:00<?, ?it/s]

In [7]:
def infer_title_length_publication(rows):
    total_active_years = 0

    for _, row in rows.iterrows():
        # Check if both dates are not null
        if pd.notnull(row['processed_date_first_issue_online']) and pd.notnull(row['processed_date_last_issue_online']):
            start_year = row['processed_date_first_issue_online'].year
            end_year = row['processed_date_last_issue_online'].year
            active_years = end_year - start_year + 1  # +1 to include both start and end year
            total_active_years += active_years

    # Apply the total active years to each row in the group
    rows['title_active_years'] = total_active_years
    return rows

tqdm.pandas(desc="Infer length of publication by title")
global_txt_file = global_txt_file.groupby('title_id').progress_apply(infer_title_length_publication)

Infer length of publication by title:   0%|          | 0/4469 [00:00<?, ?it/s]

In [8]:
def infer_missing_publisher(rows):
    publisher_name = rows['publisher_name'].unique().tolist()
    publisher_name = [x for x in publisher_name if pd.notnull(x)]

    if len(publisher_name) > 1:
        print(f"Multiple publishers found for this publication {publisher_name}")
        publisher_name = ','.join(publisher_name)
    if len(publisher_name) == 1:
        publisher_name = publisher_name[0]
    
    if len(publisher_name) == 0:
        publisher_name = np.nan
    
    rows['processed_publisher_name'] = publisher_name

    return rows

tqdm.pandas(desc="Infer missing publisher")
global_txt_file = global_txt_file.sort_values(by=['processed_date_first_issue_online'])
global_txt_file = global_txt_file.groupby('lineage').progress_apply(infer_missing_publisher)

Infer missing publisher:   0%|          | 0/2851 [00:00<?, ?it/s]

Multiple publishers found for this publication ['Centro Studi Paesi Extraeuropei (CSPE)', 'Viella SRL']
Multiple publishers found for this publication ['Cambridge University Press', 'The North American Conference on British Studies']
Multiple publishers found for this publication ['American Humor Studies Association', 'Penn State University Press']
Multiple publishers found for this publication ['Archaeological Institute of America', 'The University of Chicago Press']
Multiple publishers found for this publication ['American Statistical Association', 'Taylor & Francis, Ltd.']
Multiple publishers found for this publication ['Taylor & Francis, Ltd.', 'Association of American Geographers']
Multiple publishers found for this publication ['Wiley', 'Oxford University Press']
Multiple publishers found for this publication ['RAND Corporation', 'Wiley']
Multiple publishers found for this publication ['Stanford University Press', 'Annual Reviews']
Multiple publishers found for this publication [

In [9]:
global_txt_file[['num_first_vol_online', 'num_first_issue_online', 'num_last_issue_online', 'num_last_vol_online']].describe()

,num_first_vol_online,num_first_issue_online,num_last_issue_online,num_last_vol_online
count,3940,3756,3787,3995
unique,162,140,313,264
top,1,1,4,3
freq,2800,3202,1143,94


In [141]:
global_txt_file.lineage.nunique()

2851

In [10]:
global_txt_file.num_first_vol_online.unique()

array(['1', '13', '66', nan, '2', '12', '5', '4', '8', '7', '3', '1 (9)',
       '9', '4 (12)', '11', '14', '25', '6', '26', '19', '23', '47', '31',
       '178', '1887', '50', '1888', '17', '1890', '40', '54', '35', '56',
       '187', '28', '32', '46', '33', '24', '18', '52', '45', '76', '37',
       '15', '40 (Anno 21)', '20', '10', '16', '0', '12/13', '41', '82',
       '21', '27', '10 (44)', '36', '29', 'א', '30', '39', "א'", '22',
       '95', '5 (ANNO 42)', '116', '224', '147', '234', '48', '51', '34',
       '71', '43', '93', '38', '53', '44', '90', '111', '68', '42', 'יז',
       '69', '86', '1 (49)', '1960', 'כו', '77', '8 (56)', '61', 'ד',
       '59', '67', '64', '49', '1/2/3', '1970', '87', '123', '104', '[1]',
       '89', '78', '75', '57', '92', '65', '88', '84', '79', '282', '70',
       '63', '112', '142', '98', '136', '101', '151', '297', '62', '1989',
       '33 (81)', '430', '332', '241', '329', '27P', '152', '153', '93B',
       '29A', '114', '452', '354', '128', '

In [11]:
first_issues = global_txt_file.num_first_issue_online.unique().tolist()
first_issues_numbers = [x for x in first_issues if str(x).isdigit()]
first_issues_not_numbers = [x for x in first_issues if not str(x).isdigit()]
last_issues = global_txt_file.num_last_issue_online.unique().tolist()
last_issues_numbers = [x for x in last_issues if str(x).isdigit()]
last_issues_not_numbers = [x for x in last_issues if not str(x).isdigit()]


In [12]:
global_txt_file[(~global_txt_file.num_first_issue_online.isin(first_issues_not_numbers)) & (~global_txt_file.num_last_issue_online.isin(last_issues_not_numbers)) & (global_txt_file.num_first_vol_online.notna()) & (global_txt_file.num_last_vol_online.notna()) & (global_txt_file.processed_date_first_issue_online.notna())][['num_first_issue_online', 'num_last_issue_online', 'num_first_vol_online', 'num_last_vol_online', 'processed_date_first_issue_online', 'processed_date_last_issue_online', 'publication_title']]

,num_first_issue_online,num_last_issue_online,num_first_vol_online,num_last_vol_online,processed_date_first_issue_online,processed_date_last_issue_online,publication_title
564,1,77,1,13,1808-09-01,1814-12-31,The Belfast Monthly Magazine
3100,1,31,1,12,1815-05-01,1821-04-01,The North-American Review and Miscellaneous Jo...
411,1,6,1,218,1818-01-01,2018-12-01,Archiv für die civilistische Praxis
2038,5,134,2,44,1819-03-01,1829-12-01,"Journal des voyages, découvertes et navigation..."
896,1,30,1,3,1819-09-01,1822-06-01,The Cambro-Briton
...,...,...,...,...,...,...,...
1195,229,239,59,63,2020-06-18,2023-01-01,Desarrollo Económico
1359,1,4,75,77,2021-01-01,2023-10-01,Erdkunde
1192,1,5,58,60,2021-02-01,2023-10-01,Demography
117,4,4,68,69,2021-12-01,2022-12-01,Air & Space Power History


In [13]:
global_txt_file[global_txt_file.num_first_issue_online == '1/2'][['publication_title', 'num_first_vol_online', 'num_first_issue_online', 'num_last_issue_online', 'num_last_vol_online', 'processed_date_first_issue_online', 'processed_date_last_issue_online']]

,publication_title,num_first_vol_online,num_first_issue_online,num_last_issue_online,num_last_vol_online,processed_date_first_issue_online,processed_date_last_issue_online
65,Advocate of Peace (1847-1884),7,1/2,3,15,1847-01-01,1884-06-01
711,Bulletin de la Société de l'Histoire du Protes...,1,1/2,10,14,1852-06-01,1865-10-01
3423,Proceedings of the Cambridge Philological Society,NaN,1/2,50,NaN,1882-01-01,2004-01-01
4487,Zeitschrift für Theologie und Kirche,1,1/2,4,115,1891-01-01,2018-12-01
202,The American Journal of Semitic Languages and ...,12,1/2,4,58,1895-10-01,1941-10-01
...,...,...,...,...,...,...,...
2288,Journal of Korean Religions,1,1/2,2,13,2010-09-01,2022-10-01
1592,Gerontology & Geriatrics / גרונטולוגיה וגריאטריה,לט,1/2,3,48,2012-01-01,2021-01-01
1772,The Hungarian Historical Review,1,1/2,4,10,2012-01-01,2021-01-01
2106,Journal of Austrian Studies,45,1/2,4,53,2012-04-01,2020-12-01


In [14]:
first_issues_not_numbers

[nan,
 '1/2',
 '2/3',
 '25 (880)',
 '1896/1897',
 '1 (1)',
 '1 (15)',
 '1/4',
 '1/3',
 '1 (87)',
 "א'",
 'א',
 '11 (305)',
 '3/4',
 'א/ב',
 '[1]',
 '104/109',
 'א (א)',
 '1959/1962',
 '33/34',
 '4 (110)',
 '1/12',
 '1 (205)',
 '1965/1975',
 '2 (2)',
 '1 (8)',
 '75/76',
 '1 (35)',
 '13 (1)',
 '5/6',
 '1 (27)',
 '4SS',
 '1985/1986',
 '4S',
 '1987/1988',
 '1 (31)',
 '11/12',
 'S-5',
 '9/10',
 '23/24',
 '20/21',
 '1997/1998',
 '49/50',
 '1 (154)',
 '1-2',
 '93/94',
 '117/118']

#### Bin Publications Based on Inferred Length

##### By Lineage

In [15]:
global_txt_file['lineage_active_years_bin'] = pd.qcut(global_txt_file['lineage_active_years'], q=4, duplicates='drop', labels=False)
global_txt_file['lineage_active_years_bin'] = "binned_" + global_txt_file['lineage_active_years_bin'].astype(str) 
global_txt_file.lineage_active_years_bin.value_counts()

binned_0    1167
binned_3    1123
binned_2    1105
binned_1    1102
Name: lineage_active_years_bin, dtype: int64

In [16]:
global_txt_file['coerced_date_first_issue_online'] = pd.to_datetime(global_txt_file['date_first_issue_online'], format='%Y-%m-%d', errors='coerce')
global_txt_file['coerced_date_last_issue_online'] = pd.to_datetime(global_txt_file['date_last_issue_online'], format='%Y-%m-%d', errors='coerce')

In [17]:
lineage_numb_pubs_by_first_issue = alt.Chart(global_txt_file[['title_id', 'coerced_date_first_issue_online', 'lineage_active_years_bin', 'lineage_active_years']]).mark_bar().encode(
    x='coerced_date_first_issue_online:T',
    y='count()',
    color='lineage_active_years_bin:N'
).properties(
    width=300,
    height=150
)

lineage_years_pubs = alt.Chart(global_txt_file[['title_id', 'coerced_date_first_issue_online', 'lineage_active_years_bin', 'lineage_active_years']]).mark_bar().encode(
    x='lineage_active_years:Q',
    y='count()',
    color='lineage_active_years_bin:N'
).properties(
    width=300,
    height=150
)
lineage_years_pubs | lineage_numb_pubs_by_first_issue


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


##### By Title

In [18]:
global_txt_file['title_active_years_bin'] = pd.qcut(global_txt_file['title_active_years'], q=4, duplicates='drop', labels=False)
global_txt_file['title_active_years_bin'] = "binned_" + global_txt_file['title_active_years_bin'].astype(str) 
global_txt_file.title_active_years_bin.value_counts()

binned_0    1149
binned_1    1125
binned_2    1115
binned_3    1108
Name: title_active_years_bin, dtype: int64

In [97]:
title_numb_pubs_by_first_issue = alt.Chart(global_txt_file[['title_id', 'coerced_date_first_issue_online', 'title_active_years_bin', 'title_active_years']]).mark_bar().encode(
    x='coerced_date_first_issue_online:T',
    y='count()',
    color='title_active_years_bin:N'
).properties(
    width=300,
    height=150
)

title_years_pubs = alt.Chart(global_txt_file[['title_id', 'coerced_date_first_issue_online', 'title_active_years_bin', 'title_active_years']]).mark_bar().encode(
    x='title_active_years:Q',
    y='count()',
    color='title_active_years_bin:N'
).properties(
    width=300,
    height=150
)
title_years_pubs | title_numb_pubs_by_first_issue


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


#### Fill Missing Disciplines

In [19]:
def clean_and_count_disciplines(df):
    # Clean discipline names and count per title
    df['discipline'] = df['discipline'].apply(lambda x: [i.strip() for i in x.split('; ')] if isinstance(x, str) else x)
    df['discipline_count_per_title'] = df['discipline'].apply(lambda x: len(x) if isinstance(x, list) else 0)
    return df

# Clean disciplines and count per title
global_txt_file = clean_and_count_disciplines(global_txt_file)

In [20]:
def aggregate_disciplines(group):
    # Combine all disciplines into a single list
    all_disciplines = []
    for disciplines in group['discipline'].dropna():
        all_disciplines.extend(disciplines)

    # Remove duplicates and sort
    unique_disciplines = sorted(set(all_disciplines))
    group['discipline_count_per_lineage'] = len(unique_disciplines)
    # Join back into a single string or keep as a list
    aggregated_disciplines = '; '.join(unique_disciplines) if unique_disciplines else None
    
    # Assign the aggregated disciplines to each row in the group
    group['aggregated_disciplines'] = aggregated_disciplines

    return group

tqdm.pandas(desc="Aggregate disciplines")
# Apply the function to each lineage group
global_txt_file = global_txt_file.groupby('lineage').progress_apply(aggregate_disciplines)

Aggregate disciplines:   0%|          | 0/2851 [00:00<?, ?it/s]

In [21]:
global_txt_file['unique_title_discipline'] = global_txt_file.groupby('title_id')['aggregated_disciplines'].transform(lambda x: x.nunique() == 1)
global_txt_file['unique_lineage_discipline'] = global_txt_file.groupby('lineage')['aggregated_disciplines'].transform(lambda x: x.nunique() == 1)

In [22]:
# # Define a function to propagate non-null disciplines within each group
# def fill_missing_disciplines(rows):
#     # If there are non-null disciplines within the rows, use the first one
#     if rows['discipline'].notnull().any():
#         # Fill missing values with the first non-null discipline in the rows
#         rows['title_discipline'] = rows['discipline'].fillna(method='bfill').fillna(method='ffill')
#     return rows

# tqdm.pandas(desc="Fill in Disciplines")
# global_txt_file = global_txt_file.sort_values(by="processed_date_first_issue_online")
# global_txt_file = global_txt_file.groupby('title_id').progress_apply(fill_missing_disciplines)

In [23]:
# unique_disicipline_lineages = global_txt_file[global_txt_file['unique_discipline'] == False].lineage.unique().tolist()

# for lineage in unique_disicipline_lineages:
#     print(lineage)
#     print(global_txt_file[global_txt_file['lineage'] == lineage].discipline.unique())
#     print()

In [24]:
# global_txt_file.to_csv("../data/original_journal_datasets/jstor/cleaned_jstor_titles.csv", index=False)

In [25]:
global_txt_file.lineage_active_years.max()

489

### Load Wiki Data for Journals

In [125]:
wiki_global_txt_file = pd.read_csv("../data/original_journal_datasets/jstor/cleaned_jstor_titles_wiki.csv")
len(wiki_global_txt_file), len(global_txt_file)

(4521, 4497)

In [126]:
global_txt_file['updated_discipline_count_per_lineage'] = global_txt_file.discipline_count_per_lineage

In [127]:
def update_wiki_dataframe(wiki_global_txt_file, global_txt_file):
    cols = global_txt_file.columns.tolist()
    wiki_cols = wiki_global_txt_file.columns.tolist()

    non_existing_cols = [col for col in cols if col not in wiki_cols]

    if len(non_existing_cols)> 0:
        non_existing_wiki_cols = [col for col in wiki_cols if col not in cols]

        print(non_existing_cols)
        print(non_existing_wiki_cols)

        subset_cols = ['title_id', 'parent_publication_title_id', 'preceding_publication_title_id', 'date_first_issue_online', 'lineage', 'parent']
        merged_df = wiki_global_txt_file.merge(global_txt_file[subset_cols + non_existing_cols], on=subset_cols, how='left')
        merged_df.to_csv("../data/original_journal_datasets/jstor/cleaned_jstor_titles_wiki.csv", index=False)
    else:
        merged_df = wiki_global_txt_file
    return merged_df

In [128]:
wiki_global_txt_file = update_wiki_dataframe(wiki_global_txt_file, global_txt_file)

In [132]:
len(wiki_global_txt_file), len(global_txt_file)

(4521, 4497)

In [129]:
wiki_global_txt_file['qid'] = wiki_global_txt_file.wikidata_url.str.split('/').str[-1]

In [130]:
wiki_global_txt_file.qid.nunique()

3063

#### Infer Wiki Data Based on Lineage

In [133]:
def propagate_wiki_data_within_lineage(group):
    # Sort the group by publication date
    group = group.sort_values(by='processed_date_first_issue_online')

    # Initialize variables to store the last known values
    last_wiki_url = None
    last_wikidata_url = None
    last_wikidata_title = None

    # Iterate through the group and propagate Wikipedia and Wikidata data
    for index, row in group.iterrows():
        # Update the last known values if present
        if pd.notnull(row['wikipedia_url']):
            last_wiki_url = row['wikipedia_url']
        if pd.notnull(row['wikidata_url']):
            last_wikidata_url = row['wikidata_url']
        if pd.notnull(row['wikidata_title']):
            last_wikidata_title = row['wikidata_title']
        
        # Propagate to rows missing the data
        if last_wiki_url is not None and pd.isnull(row['wikipedia_url']):
            group.at[index, 'wikipedia_url'] = last_wiki_url
        if last_wikidata_url is not None and pd.isnull(row['wikidata_url']):
            group.at[index, 'wikidata_url'] = last_wikidata_url
        if last_wikidata_title is not None and pd.isnull(row['wikidata_title']):
            group.at[index, 'wikidata_title'] = last_wikidata_title

    return group

inferred_wiki_global_txt_file = wiki_global_txt_file.copy()
# Apply the function to each lineage group
tqdm.pandas(desc="Propagate Wiki Data")
inferred_wiki_global_txt_file = inferred_wiki_global_txt_file.groupby('lineage').progress_apply(propagate_wiki_data_within_lineage)



Propagate Wiki Data:   0%|          | 0/2872 [00:00<?, ?it/s]

In [134]:
print(f"So we have {len(inferred_wiki_global_txt_file[(inferred_wiki_global_txt_file.wikidata_title.notna()) | (inferred_wiki_global_txt_file.wikidata_url.notna()) | (inferred_wiki_global_txt_file.wikipedia_url.notna())])} titles with some form of Wiki data and {len(inferred_wiki_global_txt_file[(inferred_wiki_global_txt_file.wikidata_title.isna()) & (inferred_wiki_global_txt_file.wikidata_url.isna()) & (inferred_wiki_global_txt_file.wikipedia_url.isna())])} titles with no Wiki data.")

So we have 3329 titles with some form of Wiki data and 1192 titles with no Wiki data.


In [135]:
def has_wiki(group, column_type):
    if group['wikidata_title'].notnull().any() or group['wikidata_url'].notnull().any() or group['wikipedia_url'].notnull().any():
        group[f'{column_type}_has_wiki'] = True
    else:
        group[f'{column_type}_has_wiki'] = False
    return group

tqdm.pandas(desc="Has Wiki")
inferred_wiki_global_txt_file = inferred_wiki_global_txt_file.groupby('lineage').progress_apply(has_wiki, column_type='lineage')
inferred_wiki_global_txt_file = inferred_wiki_global_txt_file.groupby('title_id').progress_apply(has_wiki, column_type='title')

Has Wiki:   0%|          | 0/2872 [00:00<?, ?it/s]

Has Wiki:   0%|          | 0/4496 [00:00<?, ?it/s]

In [136]:
print(f"So we have {len(inferred_wiki_global_txt_file[(inferred_wiki_global_txt_file.title_has_wiki == True) | (inferred_wiki_global_txt_file.lineage_has_wiki == True)])} titles with some form of Wiki data and {len(inferred_wiki_global_txt_file[(inferred_wiki_global_txt_file.title_has_wiki == False) & (inferred_wiki_global_txt_file.lineage_has_wiki == False)])} titles with no Wiki data.")

So we have 3853 titles with some form of Wiki data and 668 titles with no Wiki data.


In [137]:
inferred_wiki_global_txt_file.to_csv("../data/original_journal_datasets/jstor/cleaned_jstor_titles_inferred_wiki.csv", index=False)